In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2023-ml-midterm-p4/submit_sample.csv
/kaggle/input/2023-ml-midterm-p4/test_data/test_2021.csv
/kaggle/input/2023-ml-midterm-p4/test_data/test_2022.csv
/kaggle/input/2023-ml-midterm-p4/train_data/train_2019.csv
/kaggle/input/2023-ml-midterm-p4/train_data/train_2016.csv
/kaggle/input/2023-ml-midterm-p4/train_data/train_2020.csv
/kaggle/input/2023-ml-midterm-p4/train_data/train_2017.csv
/kaggle/input/2023-ml-midterm-p4/train_data/train_2015.csv
/kaggle/input/2023-ml-midterm-p4/train_data/train_2018.csv


In [2]:
#랜덤시드 고정
import random
import os
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [3]:
#데이터 불러오기
train_2015 = pd.read_csv("/kaggle/input/2023-ml-midterm-p4/train_data/train_2015.csv")
train_2016 = pd.read_csv("/kaggle/input/2023-ml-midterm-p4/train_data/train_2016.csv")
train_2017 = pd.read_csv("/kaggle/input/2023-ml-midterm-p4/train_data/train_2017.csv")
train_2018 = pd.read_csv("/kaggle/input/2023-ml-midterm-p4/train_data/train_2018.csv")
train_2019 = pd.read_csv("/kaggle/input/2023-ml-midterm-p4/train_data/train_2019.csv")
train_2020 = pd.read_csv("/kaggle/input/2023-ml-midterm-p4/train_data/train_2020.csv")

test_2021 = pd.read_csv("/kaggle/input/2023-ml-midterm-p4/test_data/test_2021.csv")
test_2022 = pd.read_csv("/kaggle/input/2023-ml-midterm-p4/test_data/test_2022.csv")

submit = pd.read_csv("/kaggle/input/2023-ml-midterm-p4/submit_sample.csv")

In [4]:
# 소수점이 "." 이아니라 "," 로 되어있어 이를 바꿔줌
test_2022 = test_2022[['GDP per capita', 'Healthy Life Expectancy', 'Social support', 'Freedom to make life choices', 'Generosity', 'Corruption Perception', 'Residual error']]
for i in ['GDP per capita', 'Healthy Life Expectancy', 'Social support', 'Freedom to make life choices', 'Generosity', 'Corruption Perception', 'Residual error']:
    for j in range(len(test_2022)):
        test_2022[i][j] = '.'.join(test_2022[i][j].split(','))

# 결측치 채우기
### 문제 설명을 바탕으로 채움

In [5]:
for i in range(len(train_2018['Residual error'])):
    train_2018['Residual error'][i] = train_2018.iloc[i]['Happiness Score'] - train_2018.iloc[i][['GDP per capita', 'Healthy Life Expectancy', 'Social support', 'Freedom to make life choices', 'Generosity', 'Corruption Perception']].sum()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
for i in range(len(train_2019['Residual error'])):
    train_2019['Residual error'][i] = train_2019.iloc[i]['Happiness Score'] - train_2019.iloc[i][['GDP per capita', 'Healthy Life Expectancy', 'Social support', 'Freedom to make life choices', 'Generosity', 'Corruption Perception']].sum()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# 데이터 합치기

In [7]:
train = pd.concat([train_2015,train_2016,train_2017,train_2018,train_2019,train_2020],axis=0)
test = pd.concat([test_2021,test_2022],axis=0)

In [8]:
train = train[['GDP per capita', 'Healthy Life Expectancy', 'Social support', 'Freedom to make life choices', 'Generosity', 'Corruption Perception', 'Residual error','Happiness Score']]
test = test[['GDP per capita', 'Healthy Life Expectancy', 'Social support', 'Freedom to make life choices', 'Generosity', 'Corruption Perception', 'Residual error']]

# 독립,종속 데이터 나누기

In [9]:
x=train[['GDP per capita', 'Healthy Life Expectancy', 'Social support', 'Freedom to make life choices', 'Generosity', 'Corruption Perception', 'Residual error']]
y=train['Happiness Score']

In [10]:
#데이터 type및 null 확인하기
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 935 entries, 0 to 152
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   GDP per capita                935 non-null    float64
 1   Healthy Life Expectancy       935 non-null    float64
 2   Social support                935 non-null    float64
 3   Freedom to make life choices  935 non-null    float64
 4   Generosity                    935 non-null    float64
 5   Corruption Perception         934 non-null    float64
 6   Residual error                935 non-null    float64
dtypes: float64(7)
memory usage: 58.4 KB


In [11]:
#결측치가 있음으로 채워주겠음
from sklearn.impute import SimpleImputer
si = SimpleImputer()
x = si.fit_transform(x)

In [12]:
#scaling 해주기
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
x_st = ss.fit_transform(x)
test_st = ss.transform(test)

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [13]:
# 모델 학습 및 평가
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error
model = KNeighborsRegressor(weights='distance')
model.fit(x_st,y)
print(mean_absolute_error(y,model.predict(x_st)))

0.0


In [14]:
#submit 형태 확인
submit.head()

,id,score
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [15]:
# 제출
submit['score']=model.predict(test_st)
submit.to_csv("submit.csv",mode='w',header=True,index=False)